In [4]:
%pip install -q duckdb

Note: you may need to restart the kernel to use updated packages.


In [5]:
# If needed, install deps. Commented by default.
# %pip install pandas numpy scikit-learn duckdb matplotlib pyarrow

import os, gc, math, warnings
from pathlib import Path
import numpy as np, pandas as pd
import duckdb
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, roc_auc_score, roc_curve, confusion_matrix, ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 120)


In [6]:
%pip install -q kaggle

Note: you may need to restart the kernel to use updated packages.


In [9]:
from kaggle.api.kaggle_api_extended import KaggleApi
from pathlib import Path
import os, json

data_dir = Path('G:/Master\'s Studies/Uni WSB/4th Sem/Decision Support System/WSB-DSS/data/airbnb_seattle')
data_dir.mkdir(parents=True, exist_ok=True)

needed = [
    'listings.csv',
    'calendar.csv',
    'reviews.csv'
]

def have_csvs(d):
    return all((d / f).exists() for f in needed)

if not have_csvs(data_dir):
    print('Attempting Kaggle download...')
    
    # 1. Define the path to your JSON file
    config_path = Path('G:/Master\'s Studies/Uni WSB/4th Sem/Decision Support System/WSB-DSS/kaggle_key') / 'kaggle.json'
    
    if not config_path.exists():
        print(f"Kaggle download failed: '{config_path}' not found.")
    else:
        try:
            # 2. Read the credentials from your file
            with open(config_path, 'r') as f:
                creds = json.load(f)
            
            # 3. Set credentials *temporarily* for the API
            os.environ['KAGGLE_USERNAME'] = creds['username']
            os.environ['KAGGLE_KEY'] = creds['key']
            
            # 4. Initialize and use the Python API
            api = KaggleApi()
            api.authenticate()  # This will now succeed
            
            print('Downloading dataset to', str(data_dir))
            api.dataset_download_files(
                'airbnb/seattle', 
                path=str(data_dir), 
                unzip=True
            )
            print('Kaggle download complete.')
            
        except Exception as e:
            print(f'Kaggle download failed. Error: {e}')
else:
    print("CSV files are already present.")

print('CSVs present:', have_csvs(data_dir))


CSV files are already present.
CSVs present: True


In [10]:
import pandas as pd
from pathlib import Path

# Assumes `data_dir` already points to your Seattle Airbnb folder
files = {
    'listings': data_dir / 'listings.csv',
    'calendar': data_dir / 'calendar.csv',
    'reviews' : data_dir / 'reviews.csv'
}

# Minimal required columns to confirm we're looking at the right schema
required = {
    'listings': ['id', 'host_id', 'price'],
    'calendar': ['listing_id', 'date', 'available', 'price'],
    'reviews' : ['listing_id', 'date', 'comments']
}

def check_csv(path: Path, required_cols):
    if not path.exists():
        print(f"[MISSING] {path}")
        return False
    # Read only header to avoid heavy IO
    cols = pd.read_csv(path, nrows=0).columns.str.lower().tolist()
    missing = [c for c in required_cols if c.lower() not in cols]
    if missing:
        print(f"[CHECK] {path.name}: missing expected columns: {missing}")
        return False
    print(f"[OK] {path.name}")
    return True

all_ok = True
for k, p in files.items():
    all_ok &= check_csv(p, required[k])

# Load the data (parse date columns if present; safe even if some are missing)
listings = pd.read_csv(files['listings'])
for col in ['host_since', 'first_review', 'last_review', 'calendar_last_scraped', 'last_scraped']:
    if col in listings.columns:
        listings[col] = pd.to_datetime(listings[col], errors='coerce')

calendar = pd.read_csv(files['calendar'])
if 'date' in calendar.columns:
    calendar['date'] = pd.to_datetime(calendar['date'], errors='coerce')

reviews = pd.read_csv(files['reviews'])
if 'date' in reviews.columns:
    reviews['date'] = pd.to_datetime(reviews['date'], errors='coerce')

print('Shapes:', listings.shape, calendar.shape, reviews.shape)


[OK] listings.csv
[OK] calendar.csv
[OK] reviews.csv
Shapes: (3818, 92) (1393570, 4) (84849, 6)


In [12]:
# === Run tracking + artifact saving (DuckDB + filesystem) for regression / logistic / kmeans ===
import os, json, uuid, datetime as dt
from pathlib import Path

import duckdb
import joblib
import pandas as pd
import matplotlib.pyplot as plt

# ---------- Config ----------
DB_PATH  = Path("wsb_dss.duckdb")
ART_ROOT = Path("artifacts")
ART_ROOT.mkdir(exist_ok=True)

# ---------- DB helpers ----------
def _con():
    con = duckdb.connect(str(DB_PATH))
    con.execute("""
        CREATE TABLE IF NOT EXISTS runs (
            run_id TEXT PRIMARY KEY,
            model_type TEXT,
            started_at TIMESTAMP,
            ended_at TIMESTAMP,
            status TEXT,              -- 'running'|'succeeded'|'failed'
            params TEXT,              -- JSON string
            metrics TEXT,             -- JSON string
            dataset_id TEXT,
            notes TEXT
        );
    """)
    con.execute("""
        CREATE TABLE IF NOT EXISTS artifacts (
            run_id TEXT,
            artifact_type TEXT,       -- 'plot'|'table'|'model'|'text'
            uri TEXT,                 -- relative path
            description TEXT,
            PRIMARY KEY (run_id, artifact_type, uri)
        );
    """)
    return con

def start_run(model_type, params=None, dataset_id="airbnb_seattle"):
    run_id = f"{dt.datetime.now():%Y%m%d-%H%M%S}-{model_type}-{str(uuid.uuid4())[:8]}"
    _con().execute("INSERT INTO runs VALUES (?,?,?,?,?,?,?,?,?)",
                   [run_id, model_type, dt.datetime.now(), None, 'running',
                    json.dumps(params or {}), None, dataset_id, None])
    return run_id

def end_run(run_id, status='succeeded', metrics=None, notes=None):
    _con().execute("""
        UPDATE runs SET ended_at=?, status=?, metrics=?, notes=? WHERE run_id=?
    """, [dt.datetime.now(), status, json.dumps(metrics or {}), notes, run_id])

# --- helper: ensure run row exists when a run_id is provided externally ---
def _ensure_run_row(run_id, model_type, params=None, dataset_id="airbnb_seattle"):
    row = _con().execute("SELECT 1 FROM runs WHERE run_id=?", [run_id]).fetchone()
    if row is None:
        _con().execute(
            "INSERT INTO runs VALUES (?,?,?,?,?,?,?,?,?)",
            [run_id, model_type, dt.datetime.now(), None, 'running',
             json.dumps(params or {}), None, dataset_id, None]
        )


def log_artifact(run_id, path, artifact_type='plot', description=''):
    rel = str(Path(path))
    _con().execute("INSERT OR REPLACE INTO artifacts VALUES (?,?,?,?)",
                   [run_id, artifact_type, rel, description])

def latest_run(model_type):
    return _con().execute("""
        SELECT * FROM runs WHERE model_type=? ORDER BY ended_at DESC NULLS LAST, started_at DESC LIMIT 1
    """, [model_type]).df()

def runs_history(model_type=None):
    if model_type:
        return _con().execute("SELECT * FROM runs WHERE model_type=? ORDER BY started_at DESC", [model_type]).df()
    return _con().execute("SELECT * FROM runs ORDER BY started_at DESC").df()


In [13]:
# === Audit tables (no history) + persist helpers ============================
import json, datetime as dt
from pathlib import Path
import duckdb
import pandas as pd

# Create the two singleton audit tables (1 row per dataset_id)
def ensure_audit_tables():
    con = _con()
    con.execute("""
        CREATE TABLE IF NOT EXISTS health_checks (
            dataset_id TEXT PRIMARY KEY,
            computed_at TIMESTAMP,
            metrics TEXT            -- JSON
        );
    """)
    con.execute("""
        CREATE TABLE IF NOT EXISTS deep_dive_checks (
            dataset_id TEXT PRIMARY KEY,
            computed_at TIMESTAMP,
            metrics TEXT            -- JSON
        );
    """)
    # Auxiliary tables for detailed frames (we replace them entirely on persist)
    # We store one copy per dataset in each table via a dataset_id column.
    # Tables are created on first persist.

ensure_audit_tables()

def _df_to_table(con, df: pd.DataFrame, table_name: str, dataset_id: str):
    df = df.copy()
    df.insert(0, "dataset_id", dataset_id)
    con.execute(f"CREATE TABLE IF NOT EXISTS {table_name} AS SELECT * FROM df LIMIT 0")
    # Replace the whole table for this dataset_id
    con.execute(f"DELETE FROM {table_name} WHERE dataset_id = ?", [dataset_id])
    con.register("df", df)
    con.execute(f"INSERT INTO {table_name} SELECT * FROM df")
    con.unregister("df")

def persist_health_audit(audit: dict, dataset_id="airbnb_seattle", overwrite=False, verbose=True):
    """
    audit: dict from run_health_audit(...): {'metrics': {...}, 'tables': {...}}
    Writes a single row to health_checks; stores detailed frames in *_health_* tables.
    """
    con = _con()
    # skip if exists and not overwriting
    exists = con.execute("SELECT 1 FROM health_checks WHERE dataset_id=?", [dataset_id]).fetchone() is not None
    if exists and not overwrite:
        if verbose: print(f"[health_checks] Row already exists for dataset_id='{dataset_id}'. Skipping (overwrite=False).")
        return

    # upsert the JSON metrics row
    con.execute("DELETE FROM health_checks WHERE dataset_id=?", [dataset_id])
    con.execute(
        "INSERT INTO health_checks VALUES (?,?,?)",
        [dataset_id, dt.datetime.now(), json.dumps(audit.get("metrics", {}))]
    )

    # store detailed tables (replace per dataset_id)
    tbls = audit.get("tables", {})
    if isinstance(tbls.get("missing_listings"), pd.DataFrame):
        _df_to_table(con, tbls["missing_listings"], "detail_health_missing_listings", dataset_id)
    if isinstance(tbls.get("missing_calendar"), pd.DataFrame):
        _df_to_table(con, tbls["missing_calendar"], "detail_health_missing_calendar", dataset_id)
    if isinstance(tbls.get("missing_reviews"), pd.DataFrame):
        _df_to_table(con, tbls["missing_reviews"], "detail_health_missing_reviews", dataset_id)
    if isinstance(tbls.get("bad_avail_examples"), pd.DataFrame):
        _df_to_table(con, tbls["bad_avail_examples"], "detail_health_bad_availability", dataset_id)
    if isinstance(tbls.get("rev_consistency_sample"), pd.DataFrame):
        _df_to_table(con, tbls["rev_consistency_sample"], "detail_health_review_consistency", dataset_id)
    if isinstance(tbls.get("price_summaries"), pd.DataFrame):
        _df_to_table(con, tbls["price_summaries"], "detail_health_price_summaries", dataset_id)

    if verbose: print(f"[health_checks] Persisted for dataset_id='{dataset_id}'.")

def persist_deep_dive_audit(audit: dict, dataset_id="airbnb_seattle", overwrite=False, verbose=True):
    """
    audit: dict from run_deep_dive_audit(...): {'metrics': {...}, 'tables': {...}}
    Writes a single row to deep_dive_checks; stores frames in *_deepdive_* tables.
    """
    con = _con()
    exists = con.execute("SELECT 1 FROM deep_dive_checks WHERE dataset_id=?", [dataset_id]).fetchone() is not None
    if exists and not overwrite:
        if verbose: print(f"[deep_dive_checks] Row already exists for dataset_id='{dataset_id}'. Skipping (overwrite=False).")
        return

    con.execute("DELETE FROM deep_dive_checks WHERE dataset_id=?", [dataset_id])
    con.execute(
        "INSERT INTO deep_dive_checks VALUES (?,?,?)",
        [dataset_id, dt.datetime.now(), json.dumps(audit.get("metrics", {}))]
    )

    tbls = audit.get("tables", {})
    if isinstance(tbls.get("miss_by_avail"), pd.DataFrame):
        _df_to_table(con, tbls["miss_by_avail"], "detail_deepdive_price_missing_by_avail", dataset_id)
    if isinstance(tbls.get("occ_by_listing"), pd.DataFrame):
        _df_to_table(con, tbls["occ_by_listing"], "detail_deepdive_occupancy_by_listing", dataset_id)
    if isinstance(tbls.get("gap_summary"), pd.DataFrame):
        _df_to_table(con, tbls["gap_summary"], "detail_deepdive_gap_summary", dataset_id)
    if isinstance(tbls.get("notable_gaps"), pd.DataFrame):
        _df_to_table(con, tbls["notable_gaps"], "detail_deepdive_notable_gaps", dataset_id)
    if isinstance(tbls.get("rev_stats"), pd.DataFrame):
        _df_to_table(con, tbls["rev_stats"], "detail_deepdive_review_stats", dataset_id)
    if isinstance(tbls.get("top_neighborhoods"), pd.DataFrame):
        _df_to_table(con, tbls["top_neighborhoods"], "detail_deepdive_top_neighborhoods", dataset_id)

    if verbose: print(f"[deep_dive_checks] Persisted for dataset_id='{dataset_id}'.")


In [16]:
# === Health audit (turns your quick health check into structured outputs) ===
import numpy as np, pandas as pd

def run_health_audit(listings: pd.DataFrame, calendar: pd.DataFrame, reviews: pd.DataFrame, verbose=True):
    def money_to_float(s):
        if pd.api.types.is_numeric_dtype(s):
            return s.astype(float)
        return (s.astype(str).str.replace(r'[^0-9.\-]', '', regex=True)
                    .replace({'': np.nan, '.': np.nan}).astype(float))

    # normalize copies to avoid side-effects
    L = listings.copy()
    C = calendar.copy()
    R = reviews.copy()

    # derived columns (non-destructive for caller)
    if 'price' in L.columns and 'price_num' not in L.columns:
        L['price_num'] = money_to_float(L['price'])
    if 'price' in C.columns and 'price_num' not in C.columns:
        C['price_num'] = money_to_float(C['price'])
    if 'available' in C.columns:
        C['available_norm'] = C['available'].astype(str).str.lower().str.strip()

    # shapes
    rows_cols = {
        "listings_rows": int(L.shape[0]), "listings_cols": int(L.shape[1]),
        "calendar_rows": int(C.shape[0]), "calendar_cols": int(C.shape[1]),
        "reviews_rows":  int(R.shape[0]), "reviews_cols":  int(R.shape[1]),
    }

    # missingness (full, but we’ll store as frames)
    miss_L = L.isna().mean().sort_values(ascending=False).to_frame("missing_pct").mul(100).round(2)
    miss_C = C.isna().mean().sort_values(ascending=False).to_frame("missing_pct").mul(100).round(2)
    miss_R = R.isna().mean().sort_values(ascending=False).to_frame("missing_pct").mul(100).round(2)

    # duplicates
    dup_list = int(L['id'].duplicated().sum()) if 'id' in L.columns else None
    dup_cal = int(C.duplicated(['listing_id','date']).sum()) if {'listing_id','date'}.issubset(C.columns) else None
    rev_subset = [c for c in ['listing_id','date','reviewer_id','id'] if c in R.columns]
    dup_rev = int(R.duplicated(rev_subset).sum()) if len(rev_subset) >= 2 else None

    # referential integrity
    cal_missing = int((~C['listing_id'].isin(L['id'])).sum()) if {'listing_id'}.issubset(C.columns) and 'id' in L.columns else None
    rev_missing = int((~R['listing_id'].isin(L['id'])).sum()) if {'listing_id'}.issubset(R.columns) and 'id' in L.columns else None

    # dates (only ranges)
    def _range(df, col):
        if col in df.columns and pd.api.types.is_datetime64_any_dtype(df[col]):
            return str(df[col].min()), str(df[col].max())
        return None, None
    date_ranges = {
        "listings.host_since": _range(L,'host_since'),
        "listings.first_review": _range(L,'first_review'),
        "listings.last_review": _range(L,'last_review'),
        "listings.calendar_last_scraped": _range(L,'calendar_last_scraped'),
        "listings.last_scraped": _range(L,'last_scraped'),
        "calendar.date": _range(C,'date'),
        "reviews.date": _range(R,'date'),
    }

    # compare listings' first/last_review vs reviews min/max (counts only + sample mismatches)
    rev_span = None; comp = None
    first_mismatch = last_mismatch = None
    if {'listing_id','date'}.issubset(R.columns) and {'id','first_review','last_review'}.issubset(L.columns):
        rev_span = R.groupby('listing_id')['date'].agg(['min','max']).rename(columns={'min':'rev_min','max':'rev_max'})
        comp = L.set_index('id')[['first_review','last_review']].join(rev_span, how='left')
        first_mismatch = int(((~comp['first_review'].isna()) & (~comp['rev_min'].isna()) & (comp['first_review'] != comp['rev_min'])).sum())
        last_mismatch  = int(((~comp['last_review'].isna())  & (~comp['rev_max'].isna()) & (comp['last_review']  != comp['rev_max'])).sum())

    # price summaries
    def price_summary(s, name):
        v = pd.to_numeric(s, errors='coerce').dropna()
        if v.empty: return {"N":0}
        q = v.quantile([0.01,0.05,0.5,0.95,0.99])
        return {
            "N": int(v.size), "min": float(v.min()), "p1": float(q.iloc[0]), "p5": float(q.iloc[1]),
            "median": float(q.iloc[2]), "p95": float(q.iloc[3]), "p99": float(q.iloc[4]),
            "max": float(v.max()), "zeros": int((v==0).sum()), "negatives": int((v<0).sum())
        }
    price_summ = pd.DataFrame([
        {"source":"listings.price_num", **price_summary(L.get('price_num', pd.Series(dtype=float)), 'listings')},
        {"source":"calendar.price_num", **price_summary(C.get('price_num', pd.Series(dtype=float)), 'calendar')},
    ])

    # availability distribution & bad examples
    avail_counts = None; bad_avail_examples = pd.DataFrame()
    if 'available_norm' in C.columns:
        avail_counts = C['available_norm'].value_counts(dropna=False).to_dict()
        bad_mask = ~C['available_norm'].isin(['t','f'])
        if bad_mask.any():
            bad_avail_examples = C.loc[bad_mask, ['listing_id','date','available']].head(100)

    metrics = {
        "rows_cols": rows_cols,
        "duplicates": {"listings_id": dup_list, "calendar_pair": dup_cal, "reviews_keyed": dup_rev},
        "referential": {"calendar_not_in_listings": cal_missing, "reviews_not_in_listings": rev_missing},
        "date_ranges": date_ranges,
        "review_mismatch_counts": {"first_review_mismatch": first_mismatch, "last_review_mismatch": last_mismatch},
        "availability_counts": avail_counts,
    }

    tables = {
        "missing_listings": miss_L,
        "missing_calendar": miss_C,
        "missing_reviews":  miss_R,
        "bad_avail_examples": bad_avail_examples,
        "rev_consistency_sample": comp.head(50) if isinstance(comp, pd.DataFrame) else pd.DataFrame(),
        "price_summaries": price_summ,
    }

    if verbose:
        print("[Health] rows/cols:", rows_cols)
        print("[Health] duplicates:", metrics["duplicates"])
        print("[Health] referential:", metrics["referential"])
        print("[Health] availability counts:", avail_counts)

    return {"metrics": metrics, "tables": tables}


In [17]:
# === Deep-dive audit (structured returns) ===================================
import numpy as np, pandas as pd

def run_deep_dive_audit(listings: pd.DataFrame, calendar: pd.DataFrame, reviews: pd.DataFrame, verbose=True):
    C = calendar.copy(); L = listings.copy(); R = reviews.copy()
    # guards
    if 'available_norm' not in C.columns and 'available' in C.columns:
        C['available_norm'] = C['available'].astype(str).str.lower().str.strip()
    if 'price_num' not in C.columns and 'price' in C.columns:
        C['price_num'] = (C['price'].astype(str).str.replace(r'[^0-9.\-]','',regex=True)
                          .replace({'':'nan','.' : 'nan'}).astype(float))

    # 1) Price missingness by availability
    miss_by_avail = (
        C.assign(price_missing=C['price_num'].isna())
         .groupby('available_norm', dropna=False)
         .agg(rows=('price_missing','size'),
              missing=('price_missing','sum'),
              missing_pct=('price_missing', lambda s: 100*s.mean()))
         .reset_index()
    )

    # 2) Occupancy per listing
    C['booked'] = C['available_norm'].eq('f')
    occ_by_listing = C.groupby('listing_id', as_index=True)['booked'].mean().to_frame('occupancy')

    # 3) Date contiguity
    span = (C.groupby('listing_id')
              .agg(min_date=('date','min'),
                   max_date=('date','max'),
                   n_days=('date','nunique')))
    span['expected_days'] = (span['max_date'] - span['min_date']).dt.days + 1
    span['gaps'] = span['expected_days'] - span['n_days']
    gap_summary = span['gaps'].describe(percentiles=[.5,.9,.99]).to_frame(name='gaps').T.round(3)
    notable_gaps = span[span['gaps']>0].sort_values('gaps', ascending=False).head(50)

    # 4) Reviews distribution
    rev_counts = R.groupby('listing_id').size().rename('n_reviews')
    rev_stats = rev_counts.describe(percentiles=[.5,.9,.99]).to_frame().T

    # share of listings with ≥1 review
    share_with_review = None
    if 'id' in L.columns:
        share_with_review = float((L[['id']].merge(rev_counts, left_on='id', right_index=True, how='left')
                                   ['n_reviews'].fillna(0)>0).mean())

    # 5) Median prices
    med_listing_price  = float(pd.to_numeric(L.get('price_num', pd.Series(dtype=float)), errors='coerce').median(skipna=True)) \
                         if 'price_num' in L.columns else None
    med_calendar_price = float(pd.to_numeric(C.get('price_num', pd.Series(dtype=float)), errors='coerce').median(skipna=True)) \
                         if 'price_num' in C.columns else None

    # 6) Top neighborhoods
    neigh_col = next((c for c in ['neighbourhood_cleansed','neighbourhood','neighbourhood_group_cleansed'] if c in L.columns), None)
    top_neighborhoods = L[neigh_col].value_counts().head(10).to_frame('count') if neigh_col else pd.DataFrame()

    metrics = {
        "occupancy": {
            "N": int(occ_by_listing.shape[0]),
            "mean": float(occ_by_listing['occupancy'].mean()),
            "median": float(occ_by_listing['occupancy'].median()),
            "p10": float(occ_by_listing['occupancy'].quantile(0.10)),
            "p90": float(occ_by_listing['occupancy'].quantile(0.90)),
        },
        "gaps_max": int(span['gaps'].max()),
        "share_with_review": share_with_review,
        "median_prices": {"listings": med_listing_price, "calendar": med_calendar_price},
        "neigh_col": neigh_col,
    }

    tables = {
        "miss_by_avail": miss_by_avail,
        "occ_by_listing": occ_by_listing.reset_index(),
        "gap_summary": gap_summary,
        "notable_gaps": notable_gaps.reset_index(),
        "rev_stats": rev_stats,
        "top_neighborhoods": top_neighborhoods.reset_index().rename(columns={neigh_col:'neighborhood'}) if not top_neighborhoods.empty else pd.DataFrame(),
    }

    if verbose:
        print("[DeepDive] occupancy (mean, median):", metrics["occupancy"]["mean"], metrics["occupancy"]["median"])
        print("[DeepDive] gaps max:", metrics["gaps_max"])

    return {"metrics": metrics, "tables": tables}


In [18]:
# === Run & persist audits (no history) ======================================
# Health audit
health = run_health_audit(listings, calendar, reviews, verbose=True)
persist_health_audit(health, dataset_id="airbnb_seattle", overwrite=False)

# Deep-dive audit
deep = run_deep_dive_audit(listings, calendar, reviews, verbose=True)
persist_deep_dive_audit(deep, dataset_id="airbnb_seattle", overwrite=False)

# Inspect what’s stored
print(_con().execute("SELECT * FROM health_checks").df())
print(_con().execute("SELECT * FROM deep_dive_checks").df())

[Health] rows/cols: {'listings_rows': 3818, 'listings_cols': 93, 'calendar_rows': 1393570, 'calendar_cols': 6, 'reviews_rows': 84849, 'reviews_cols': 6}
[Health] duplicates: {'listings_id': 0, 'calendar_pair': 0, 'reviews_keyed': 0}
[Health] referential: {'calendar_not_in_listings': 0, 'reviews_not_in_listings': 0}
[Health] availability counts: {'t': 934542, 'f': 459028}
[health_checks] Persisted for dataset_id='airbnb_seattle'.
[DeepDive] occupancy (mean, median): 0.32938998399793334 0.15616438356164383
[DeepDive] gaps max: 0
[deep_dive_checks] Persisted for dataset_id='airbnb_seattle'.
       dataset_id                computed_at  \
0  airbnb_seattle 2025-11-12 01:24:14.417738   

                                             metrics  
0  {"rows_cols": {"listings_rows": 3818, "listing...  
       dataset_id                computed_at  \
0  airbnb_seattle 2025-11-12 01:24:17.641173   

                                             metrics  
0  {"occupancy": {"N": 3818, "mean": 0.3293899

In [ ]:
# === FUNCTION: train_regression_hgb (no run_id dependency) ===
# Trains HGB on log(price), builds same features, optionally plots, and optionally saves plots
# to a timestamped subfolder if save_dir is provided. Returns NO run_id.

import numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.inspection import permutation_importance

def train_regression_hgb(
    listings_df: pd.DataFrame,
    reviews_df: pd.DataFrame = None,
    *,
    test_size: float = 0.2,
    random_state: int = 42,
    plot: bool = True,
    save_dir: str | Path | None = None,
    top_k_importances: int = 20,
):
    """
    Train a single HGB regression on log(price).

    Returns a dict:
      {
        'model': fitted sklearn Pipeline,
        'metrics': {'mae':..., 'rmse':..., 'r2_log':...},
        'feat_importance': pd.Series (descending),
        'test_predictions': pd.DataFrame(['id','y_true','y_pred']),
        'artifacts': {'plots': [paths...]}  # if save_dir is provided
      }
    """
    df = listings_df.copy()

    # --- Helpers ---
    CITY_LAT, CITY_LON = 47.6062, -122.3321
    SPACE_NEEDLE = (47.6205, -122.3493)
    PIKE_PLACE   = (47.6094, -122.3421)

    def haversine_km(lat1, lon1, lat2, lon2):
        R = 6371.0
        lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
        dlat, dlon = lat2 - lat1, lon2 - lon1
        a = np.sin(dlat/2.0)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2.0)**2
        return 2*R*np.arcsin(np.sqrt(a))

    def tf_to_int(s):
        return s.astype(str).str.lower().map({'t':1, 'f':0})

    def has_amenity(series, keyword):
        return series.fillna('').str.lower().str.contains(keyword.lower()).astype(int)

    # Ensure numeric price exists
    if 'price_num' not in df.columns and 'price' in df.columns:
        df['price_num'] = (df['price'].astype(str)
                           .str.replace(r'[^0-9.\-]', '', regex=True)
                           .replace({'': np.nan, '.': np.nan})
                           .astype(float))

    # Target
    df = df[df['price_num'].notna()].copy()
    df['target_price'] = df['price_num']
    df['log_price'] = np.log1p(df['target_price'])

    # Review count join (leak-free)
    if reviews_df is not None:
        rev_counts = reviews_df.groupby('listing_id').size().rename('n_reviews_actual')
        df = df.merge(rev_counts, left_on='id', right_index=True, how='left')
    else:
        df['n_reviews_actual'] = np.nan
    df['n_reviews_actual'] = df['n_reviews_actual'].fillna(0)

    # Geo features
    if {'latitude','longitude'}.issubset(df.columns):
        df['dist_to_center_km']      = haversine_km(df['latitude'], df['longitude'], CITY_LAT, CITY_LON)
        df['dist_to_spaceneedle_km'] = haversine_km(df['latitude'], df['longitude'], *SPACE_NEEDLE)
        df['dist_to_pikeplace_km']   = haversine_km(df['latitude'], df['longitude'], *PIKE_PLACE)

    # Percent strings → numeric
    for pct_col in ['host_response_rate','host_acceptance_rate']:
        if pct_col in df.columns:
            df[pct_col + '_num'] = (df[pct_col].astype(str).str.rstrip('%')
                                    .replace({'nan':np.nan}).astype(float))

    # Amenities: count + flags
    if 'amenities' in df.columns:
        df['amenities_count'] = df['amenities'].fillna('').str.count(',') + df['amenities'].notna().astype(int)
        for key in ['wifi','kitchen','parking','washer','dryer']:
            df[f'amenity_{key}'] = has_amenity(df['amenities'], key)

    # Text lengths
    for text_col, newcol in [('description','desc_len'), ('name','name_len'), ('summary','summary_len')]:
        if text_col in df.columns:
            df[newcol] = df[text_col].fillna('').str.len()

    # Host tenure + booleans
    if {'host_since','last_scraped'}.issubset(df.columns):
        df['host_since_days'] = (df['last_scraped'] - df['host_since']).dt.days
    for c in ['host_is_superhost','instant_bookable','host_has_profile_pic','host_identity_verified']:
        if c in df.columns:
            df[c + '_bin'] = tf_to_int(df[c])

    # Feature lists
    num_candidates = [
        'accommodates','bathrooms','bedrooms','beds',
        'minimum_nights','maximum_nights',
        'availability_365','number_of_reviews','n_reviews_actual',
        'review_scores_rating','review_scores_cleanliness','review_scores_communication',
        'review_scores_location','review_scores_value','review_scores_accuracy','review_scores_checkin',
        'host_listings_count','host_total_listings_count',
        'host_response_rate_num','host_acceptance_rate_num',
        'amenities_count','amenity_wifi','amenity_kitchen','amenity_parking','amenity_washer','amenity_dryer',
        'desc_len','name_len','summary_len',
        'host_since_days','dist_to_center_km','dist_to_spaceneedle_km','dist_to_pikeplace_km',
        'host_is_superhost_bin','instant_bookable_bin','host_has_profile_pic_bin','host_identity_verified_bin'
    ]
    cat_candidates = ['property_type','room_type','bed_type','neighbourhood_cleansed','cancellation_policy']

    numeric_features = [c for c in num_candidates    if c in df.columns]
    categorical_features = [c for c in cat_candidates if c in df.columns]

    # Coerce numerics
    for c in numeric_features:
        df[c] = pd.to_numeric(df[c], errors='coerce')

    # Split
    X = df[numeric_features + categorical_features].copy()
    y_log   = df['log_price'].copy()
    y_price = df['target_price'].copy()
    ids = df.get('id', pd.Series(range(len(df))))

    X_train, X_test, y_train_log, y_test_log, y_train_price, y_test_price, id_train, id_test = train_test_split(
        X, y_log, y_price, ids, test_size=test_size, random_state=random_state
    )

    # Preprocess
    try:
        ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore', min_frequency=0.01)
    except TypeError:
        ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')

    pre = ColumnTransformer([
        ('num', SimpleImputer(strategy='median'), numeric_features),
        ('cat', Pipeline([('impute', SimpleImputer(strategy='most_frequent')), ('ohe', ohe)]), categorical_features)
    ], remainder='drop')

    # Model
    hgb = Pipeline([
        ('pre', pre),
        ('model', HistGradientBoostingRegressor(
            max_iter=400, learning_rate=0.06, max_leaf_nodes=31, min_samples_leaf=20,
            random_state=random_state
        ))
    ])

    # Fit
    hgb.fit(X_train, y_train_log)

    # Evaluate
    pred_log   = hgb.predict(X_test)
    pred_price = np.expm1(pred_log)
    mae  = mean_absolute_error(y_test_price, pred_price)
    rmse = mean_squared_error(y_test_price, pred_price, squared=False)
    r2lg = r2_score(y_test_log, pred_log)
    metrics = {'mae': float(mae), 'rmse': float(rmse), 'r2_log': float(r2lg)}
    print(f"[HGB] MAE=${mae:,.2f}  RMSE=${rmse:,.2f}  R²(log)={r2lg:.3f}")

    # Permutation importance
    try:
        feat_names = hgb.named_steps['pre'].get_feature_names_out()
    except Exception:
        if categorical_features:
            ohe_step = hgb.named_steps['pre'].named_transformers_['cat'].named_steps['ohe']
            try:
                cat_names = ohe_step.get_feature_names_out(categorical_features)
            except Exception:
                cat_names = np.array(categorical_features)
            feat_names = np.r_[numeric_features, cat_names]
        else:
            feat_names = np.array(numeric_features)

    pre_X_test = hgb.named_steps['pre'].transform(X_test)
    if hasattr(pre_X_test, "toarray"):
        pre_X_test = pre_X_test.toarray()

    pi = permutation_importance(
        hgb.named_steps['model'], pre_X_test, y_test_log,
        n_repeats=5, random_state=random_state, n_jobs=-1
    )
    pi_mean = pd.Series(pi.importances_mean, index=feat_names).sort_values(ascending=False)

    # Predictions table
    preds_df = pd.DataFrame({
        'id': np.array(id_test),
        'y_true': np.array(y_test_price),
        'y_pred': np.array(pred_price)
    })

    # Plots (optional)
    saved_plots = []
    if plot or save_dir:
        # 1) Pred vs Actual
        fig1 = plt.figure()
        mx = float(max(preds_df['y_true'].max(), preds_df['y_pred'].max()))
        plt.scatter(preds_df['y_true'], preds_df['y_pred'], s=9, alpha=0.5)
        plt.plot([0, mx],[0, mx])
        plt.xlabel("Actual price"); plt.ylabel("Predicted price"); plt.title("Predicted vs Actual (HGB)")
        plt.tight_layout()

        # 2) Residuals hist
        fig2 = plt.figure()
        resid = preds_df['y_true'] - preds_df['y_pred']
        plt.hist(resid, bins=50, alpha=0.9)
        plt.xlabel("Residual ($)"); plt.ylabel("Count"); plt.title("Residuals (HGB)")
        plt.tight_layout()

        if save_dir:
            ts  = datetime.now().strftime("%Y%m%d-%H%M%S")
            out = Path(save_dir) / f"run_{ts}"
            out.mkdir(parents=True, exist_ok=True)
            p1 = out / "pred_vs_actual.png"; p2 = out / "residuals_hist.png"
            fig1.savefig(p1, dpi=120, bbox_inches='tight'); fig2.savefig(p2, dpi=120, bbox_inches='tight')
            saved_plots += [str(p1), str(p2)]

    # Console: top-k features
    top = pi_mean.head(top_k_importances).round(4)
    print("\n[Permutation importance] Top features:")
    print(top.to_string())

    return {
        'model': hgb,
        'metrics': metrics,
        'feat_importance': pi_mean,
        'test_predictions': preds_df,
        'artifacts': {'plots': saved_plots}
    }


: 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    roc_auc_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
)
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


def _make_preprocessor(numeric_features_in, categorical_features_in, scale_numeric=True):
    # numeric pipeline
    num_steps = [("impute", SimpleImputer(strategy="median"))]
    if scale_numeric:
        num_steps.append(("scale", StandardScaler()))
    num_pipe = Pipeline(num_steps)

    # categorical pipeline with safe OneHotEncoder
    try:
        ohe = OneHotEncoder(
            handle_unknown="ignore",
            sparse_output=False,
            min_frequency=0.01,
        )
    except TypeError:
        # older sklearn: no sparse_output/min_frequency
        ohe = OneHotEncoder(
            handle_unknown="ignore",
            sparse=False,
        )

    cat_pipe = Pipeline(
        [
            ("impute", SimpleImputer(strategy="most_frequent")),
            ("ohe", ohe),
        ]
    )

    pre = ColumnTransformer(
        transformers=[
            ("num", num_pipe, numeric_features_in),
            ("cat", cat_pipe, categorical_features_in),
        ],
        remainder="drop",
    )

    return pre


# === Logistic Regression (price-bucket) – class-balanced, 'saga' solver, metrics returned ===
def run_logistic_price_bucket(
    df_in=None,
    numeric_features_in=None,
    categorical_features_in=None,
    price_quantile=0.75,
    test_size=0.2,
    random_state=42,
):
    import numpy as np, pandas as pd, matplotlib.pyplot as plt
    from sklearn.model_selection import train_test_split
    from sklearn.pipeline import Pipeline
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import (accuracy_score, classification_report, roc_auc_score,
                                 confusion_matrix, ConfusionMatrixDisplay, precision_recall_fscore_support)

    # --- resolve inputs ---
    if df_in is None:
        if "df" not in globals():
            raise ValueError("No df supplied and global 'df' not found. Run the regression feature cell first.")
        df_local = df.copy()
    else:
        df_local = df_in.copy()

    if numeric_features_in is None:
        if "numeric_features" not in globals():
            raise ValueError("numeric_features not available; pass explicitly or run regression feature cell.")
        numeric_features_in = numeric_features
    if categorical_features_in is None:
        if "categorical_features" not in globals():
            raise ValueError("categorical_features not available; pass explicitly or run regression feature cell.")
        categorical_features_in = categorical_features

    if "target_price" not in df_local.columns:
        raise ValueError("df must contain 'target_price' (from regression prep).")

    # target
    df_local = df_local[df_local["target_price"].notna()].copy()
    threshold = df_local["target_price"].quantile(price_quantile)
    df_local["is_high_price"] = (df_local["target_price"] >= threshold).astype(int)

    X = df_local[numeric_features_in + categorical_features_in]
    y = df_local["is_high_price"]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )

    pre = _make_preprocessor(numeric_features_in, categorical_features_in, scale_numeric=True)

    clf = Pipeline([
        ("pre", pre),
        ("model", LogisticRegression(
            solver="saga",            # robust with many OHE cols
            penalty="l2",
            C=1.0,
            class_weight="balanced", # handle 75/25-ish split cleanly
            max_iter=2000,
            n_jobs=-1                # ignored by some solvers; harmless
        )),
    ])

    clf.fit(X_train, y_train)

    y_pred  = clf.predict(X_test)
    y_proba = clf.predict_proba(X_test)[:, 1]

    acc = accuracy_score(y_test, y_pred)
    roc = roc_auc_score(y_test, y_proba)
    p, r, f1, _ = precision_recall_fscore_support(y_test, y_pred, average="binary", zero_division=0)

    print(f"High-price threshold (q={price_quantile:.2f}): ${threshold:,.0f}")
    print(f"Accuracy: {acc:.3f} | ROC-AUC: {roc:.3f} | F1: {f1:.3f} (P={p:.3f}, R={r:.3f})")
    print("\nClassification report:")
    print(classification_report(y_test, y_pred, zero_division=0))

    cm = confusion_matrix(y_test, y_pred)
    ConfusionMatrixDisplay(cm, display_labels=["low", "high"]).plot()
    plt.title("High- vs low-price listings")
    plt.tight_layout()

    metrics = {"threshold": float(threshold), "accuracy": float(acc), "roc_auc": float(roc),
               "precision": float(p), "recall": float(r), "f1": float(f1)}

    return clf, metrics



# === KMeans clustering – safe display, silhouette guard, tidy returns ===
def run_kmeans_clusters(
    df_in=None,
    numeric_features_in=None,
    n_clusters=5,
    random_state=42,
):
    import numpy as np, pandas as pd
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler
    from sklearn.impute import SimpleImputer
    from sklearn.cluster import KMeans
    from sklearn.metrics import silhouette_score
    try:
        from IPython.display import display
    except Exception:
        display = print  # fallback

    if df_in is None:
        if "df" not in globals():
            raise ValueError("No df supplied and global 'df' not found. Run the regression feature cell first.")
        df_local = df.copy()
    else:
        df_local = df_in.copy()

    if numeric_features_in is None:
        if "numeric_features" not in globals():
            raise ValueError("numeric_features not available; pass explicitly or run regression feature cell.")
        numeric_features_in = numeric_features

    df_local = df_local[df_local["target_price"].notna()].copy()
    X = df_local[numeric_features_in]

    km_pipe = Pipeline([
        ("impute", SimpleImputer(strategy="median")),
        ("scale", StandardScaler()),
        ("kmeans", KMeans(n_clusters=n_clusters, random_state=random_state, n_init=10)),
    ])

    labels = km_pipe.fit_predict(X)
    df_clusters = df_local.copy()
    df_clusters["cluster"] = labels

    # Silhouette (guard: need at least 2 clusters present)
    sil = np.nan
    if len(np.unique(labels)) > 1:
        X_scaled = km_pipe.named_steps["scale"].transform(km_pipe.named_steps["impute"].transform(X))
        sil = silhouette_score(X_scaled, labels)
        print(f"Silhouette score (k={n_clusters}): {sil:.3f}")
    else:
        print(f"Silhouette score skipped: only one cluster label found for k={n_clusters}.")

    cluster_summary = (
        df_clusters.groupby("cluster")["target_price"]
        .agg(count="size", mean_price="mean", median_price="median")
        .sort_values("mean_price")
    )
    print("\nCluster summary by target_price:")
    display(cluster_summary)

    preview_cols = ["id", "target_price", "cluster"]
    preview_cols = [c for c in preview_cols if c in df_clusters.columns]
    preview = df_clusters[preview_cols].head(20)

    metrics = {"silhouette": None if np.isnan(sil) else float(sil)}
    return km_pipe, preview, cluster_summary, metrics


: 

In [ ]:
# === One-and-done calendar forecaster: audit + train + evaluate + future forecast ===
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pathlib import Path
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

def run_calendar_forecast(calendar_df=None, data_dir=None, horizon=28, future_days=14,
                          plot=True, verbose=True):
    """
    Time-series demand forecast from Airbnb calendar.
    - Uses booked proxy: available == 'f'
    - Trains HGB on booked_share with time features + lags/rolls
    - Evaluates vs seasonal naive (t-7)
    - Rolls forward for `future_days` beyond the last date

    Returns dict with:
      model, daily (raw city series), features (with lags), holdout_df, future_df, metrics
    """

    # ---- Load ----
    if calendar_df is None:
        paths = []
        if data_dir is not None: paths.append(Path(data_dir) / 'calendar.csv')
        paths.append(Path('calendar.csv'))
        cal_path = next((p for p in paths if p.exists()), None)
        if cal_path is None:
            raise FileNotFoundError("calendar.csv not found. Pass `calendar_df` or set `data_dir`.")
        cal = pd.read_csv(cal_path)
    else:
        cal = calendar_df.copy()

    # ---- Audit ----
    cal.columns = cal.columns.str.lower()
    for col in ['listing_id','date','available']:
        if col not in cal.columns:
            raise ValueError(f"Required column missing: {col}")
    cal['date'] = pd.to_datetime(cal['date'], errors='coerce')
    cal['available'] = cal['available'].astype(str).str.strip().str.lower()

    if verbose:
        print("CALENDAR shape:", cal.shape)
        print("Missing (%):\n", (cal[['listing_id','date','available']].isna().mean()*100).round(2))
        print("Duplicates (listing_id,date):", int(cal.duplicated(['listing_id','date']).sum()))
        print("Date range:", cal['date'].min(), "→", cal['date'].max())

    # ---- Aggregate to daily city series ----
    cal['booked'] = cal['available'].eq('f')
    daily = (cal.groupby('date', as_index=True, observed=True)
               .agg(booked_count=('booked','sum'),
                    total_listings=('listing_id','nunique')))
    daily['booked_share'] = daily['booked_count'] / daily['total_listings']
    daily = daily.asfreq('D')  # enforce daily frequency (your data is already contiguous)

    if verbose:
        print("\nDaily head:\n", daily.head())
        print("\nBooked_share stats:\n", daily['booked_share'].describe().round(3))

    # ---- Feature engineering ----
    df = daily.copy()
    idx = df.index
    dow = idx.dayofweek.values
    df['dow_sin'] = np.sin(2*np.pi*dow/7.0)
    df['dow_cos'] = np.cos(2*np.pi*dow/7.0)
    df['month']   = idx.month
    df['t']       = (idx - idx.min()).days.astype(int)   # linear trend
    try:
        from pandas.tseries.holiday import USFederalHolidayCalendar
        hol = USFederalHolidayCalendar().holidays(start=idx.min(), end=idx.max())
        df['is_holiday'] = df.index.isin(hol).astype(int)
    except Exception:
        df['is_holiday'] = 0

    for L in [1, 7, 14, 21, 28]:
        df[f'lag_{L}'] = df['booked_share'].shift(L)
    df['roll7']  = df['booked_share'].rolling(7,  min_periods=5).mean()
    df['roll28'] = df['booked_share'].rolling(28, min_periods=10).mean()

    dfm = df.dropna().copy()
    if len(dfm) <= horizon + 1:
        raise ValueError("Not enough history after lags for this horizon.")

    # ---- Train / test split (last `horizon` days) ----
    train = dfm.iloc[:-horizon]
    test  = dfm.iloc[-horizon:]
    feats = ['dow_sin','dow_cos','month','t','is_holiday',
             'lag_1','lag_7','lag_14','lag_21','lag_28','roll7','roll28']
    X_tr, y_tr = train[feats], train['booked_share']
    X_te, y_te = test[feats],  test['booked_share']

    # ---- Model ----
    model = HistGradientBoostingRegressor(
        max_iter=700, learning_rate=0.05, max_leaf_nodes=31,
        min_samples_leaf=8, random_state=42
    ).fit(X_tr, y_tr)

    # Predictions on holdout (share & count)
    yhat_share = model.predict(X_te).clip(0, 1)
    yhat_cnt   = (yhat_share * test['total_listings']).round()
    ytrue_cnt  = test['booked_count']

    # Seasonal naive
    ynaive_share = test['lag_7'].values
    ynaive_cnt   = (ynaive_share * test['total_listings']).round()

    # Metrics
    def mape(y, yhat):
        y = np.asarray(y, float); yhat = np.asarray(yhat, float)
        return np.mean(np.abs((y - yhat) / np.clip(y, 1e-6, None))) * 100

    metrics = {
        'mae_model':  float(mean_absolute_error(ytrue_cnt, yhat_cnt)),
        'rmse_model': float(mean_squared_error(ytrue_cnt, yhat_cnt, squared=False)),
        'mape_model': float(mape(ytrue_cnt, yhat_cnt)),
        'mae_naive':  float(mean_absolute_error(ytrue_cnt, ynaive_cnt)),
        'rmse_naive': float(mean_squared_error(ytrue_cnt, ynaive_cnt, squared=False)),
        'mape_naive': float(mape(ytrue_cnt, ynaive_cnt)),
    }
    if verbose:
        print(f"\nHoldout ({horizon}d) — "
              f"Model MAE={metrics['mae_model']:.2f} RMSE={metrics['rmse_model']:.2f} MAPE={metrics['mape_model']:.2f}% | "
              f"Naive MAE={metrics['mae_naive']:.2f} RMSE={metrics['rmse_naive']:.2f} MAPE={metrics['mape_naive']:.2f}%")

    # ---- Plots: train/test + share line (clean date labels) ----
    if plot:
        fig, ax = plt.subplots()
        ax.plot(train.index, train['booked_count'], label='Train actual', linewidth=1)
        ax.plot(test.index,  test['booked_count'],  label='Test actual', linewidth=2)
        ax.plot(test.index,  yhat_cnt,              label='Model forecast', linewidth=2)
        ax.plot(test.index,  ynaive_cnt,            label='Naive (t-7)', linestyle='--')
        ax.set_ylim(bottom=0)
        ax.set_title("Daily booked (city-level)")
        ax.set_ylabel("Booked count")
        ax.set_xlabel("Date")
        ax.legend()
        try:
            locator = mdates.AutoDateLocator(minticks=4, maxticks=8)
            ax.xaxis.set_major_locator(locator)
            ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(locator))
        except Exception:
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
        for lbl in ax.get_xticklabels():
            lbl.set_rotation(30); lbl.set_ha('right')
        fig.tight_layout()

        fig2, ax2 = plt.subplots()
        ax2.plot(df.index, df['booked_share'])
        ax2.set_ylim(bottom=0)
        ax2.set_title("Booked share over time")
        ax2.set_ylabel("Share of listings booked")
        ax2.set_xlabel("Date")
        try:
            locator = mdates.AutoDateLocator(minticks=6, maxticks=10)
            ax2.xaxis.set_major_locator(locator)
            ax2.xaxis.set_major_formatter(mdates.ConciseDateFormatter(locator))
        except Exception:
            ax2.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
        for lbl in ax2.get_xticklabels():
            lbl.set_rotation(30); lbl.set_ha('right')
        fig2.tight_layout()

    # ---- Future forecast (roll-forward) ----
    future_rows = []
    series = dfm['booked_share'].copy()                 # last known target series (with lags populated)
    last_date, min_date = series.index[-1], series.index.min()
    supply = int(dfm['total_listings'].iloc[-1])        # keep last observed supply
    try:
        from pandas.tseries.holiday import USFederalHolidayCalendar
        hol = set(USFederalHolidayCalendar().holidays(start=min_date, end=last_date + pd.Timedelta(days=future_days)))
    except Exception:
        hol = set()

    for i in range(1, future_days + 1):
        d = last_date + pd.Timedelta(days=i)
        dow = d.dayofweek
        feat_row = {
            'dow_sin': np.sin(2*np.pi*dow/7.0),
            'dow_cos': np.cos(2*np.pi*dow/7.0),
            'month':   d.month,
            't':       (d - min_date).days,
            'is_holiday': int(d in hol),
            'lag_1':  series.iloc[-1],
            'lag_7':  series.iloc[-7]  if len(series) >= 7  else series.iloc[-1],
            'lag_14': series.iloc[-14] if len(series) >= 14 else series.iloc[-1],
            'lag_21': series.iloc[-21] if len(series) >= 21 else series.iloc[-1],
            'lag_28': series.iloc[-28] if len(series) >= 28 else series.iloc[-1],
            'roll7':  series.tail(7).mean(),
            'roll28': series.tail(28).mean(),
        }
        Xf = pd.DataFrame([feat_row], index=[d])
        yhat_share_f = float(np.clip(model.predict(Xf)[0], 0, 1))
        series.loc[d] = yhat_share_f
        future_rows.append({'date': d,
                            'pred_booked_share': yhat_share_f,
                            'pred_booked_count': round(yhat_share_f * supply)})
    future_df = pd.DataFrame(future_rows).set_index('date')

    if plot:
        fig3, ax3 = plt.subplots()
        tail = dfm.tail(90)
        ax3.plot(tail.index, tail['booked_count'], label='Recent actual', linewidth=1)
        ax3.plot(future_df.index, future_df['pred_booked_count'], label=f'Forecast (+{future_days}d)', linewidth=2)
        ax3.set_ylim(bottom=0)
        ax3.set_title(f"Forecast: booked count (next {future_days} days)")
        ax3.set_ylabel("Booked count")
        ax3.set_xlabel("Date")
        ax3.legend()
        try:
            locator = mdates.AutoDateLocator(minticks=4, maxticks=8)
            ax3.xaxis.set_major_locator(locator)
            ax3.xaxis.set_major_formatter(mdates.ConciseDateFormatter(locator))
        except Exception:
            ax3.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
        for lbl in ax3.get_xticklabels():
            lbl.set_rotation(30); lbl.set_ha('right')
        fig3.tight_layout()

    # ---- Build holdout table ----
    holdout_df = pd.DataFrame({
        'actual_booked': ytrue_cnt.values,
        'model_forecast': yhat_cnt.values,
        'naive_t7': ynaive_cnt.values
    }, index=test.index)

    return {
        'model': model,
        'daily': daily,
        'features': dfm,
        'holdout_df': holdout_df,
        'future_df': future_df,
        'metrics': metrics
    }



: 

In [ ]:
# === Multilingual review sentiment (ONE cell: audit → score → summarize → plots) ===
# - Uses a multilingual transformer (CardiffNLP XLM-RoBERTa) for 3-way sentiment
# - Falls back to VADER only if transformers/torch aren't available
# - Returns compact artifacts you can reuse in the UI

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

def run_review_sentiment_nlp(reviews_df=None, data_dir=None,
                             model_name="cardiffnlp/twitter-xlm-roberta-base-sentiment",
                             batch_size=64, max_len=256, min_chars=5, plot=True, verbose=True):
    """
    Scores Airbnb reviews as positive / neutral / negative using a multilingual model.
    Returns:
      {
        'reviews_scored': DataFrame[listening_id, date, sentiment_score, label, p_pos, p_neu, p_neg, comments_clean],
        'listing_summary': per-listing % split + avg score,
        'monthly_trend': monthly mean sentiment_score,
        'overall_pct': overall % split
      }
    """
    # ---------- Load ----------
    if reviews_df is None:
        paths = []
        if data_dir is not None: paths.append(Path(data_dir) / 'reviews.csv')
        paths.append(Path('reviews.csv'))
        rv_path = next((p for p in paths if p.exists()), None)
        if rv_path is None:
            raise FileNotFoundError("reviews.csv not found. Pass `reviews_df` or set `data_dir`.")
        reviews = pd.read_csv(rv_path)
    else:
        reviews = reviews_df.copy()

    # ---------- Basic hygiene ----------
    reviews.columns = reviews.columns.str.lower()
    need = {'listing_id','date','comments'}
    missing = need.difference(reviews.columns)
    if missing:
        raise ValueError(f"Required columns missing from reviews: {missing}")

    reviews['date'] = pd.to_datetime(reviews['date'], errors='coerce')
    before = len(reviews)
    reviews = reviews.dropna(subset=['listing_id','date','comments'])
    reviews = reviews[reviews['comments'].astype(str).str.len() >= min_chars].copy()
    # conservative de-dup
    dupe_keys = [c for c in ['listing_id','date','reviewer_id','comments'] if c in reviews.columns]
    if len(dupe_keys) >= 2:
        reviews = reviews.drop_duplicates(subset=dupe_keys)

    # light cleaning
    reviews['comments_clean'] = (reviews['comments'].astype(str)
                                 .str.replace(r'\s+', ' ', regex=True)
                                 .str.strip())

    if verbose:
        print(f"REVIEWS shape: {len(reviews)} rows (dropped {before-len(reviews)})")
        print("Missingness (%):")
        print((reviews[['listing_id','date','comments_clean']].isna().mean()*100).round(2).rename('pct').to_string())
        print("Date range:", reviews['date'].min(), "→", reviews['date'].max())

    # ---------- Sentiment model (multilingual) ----------
    use_transformer = True
    try:
        from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
        import torch  # required backend
    except Exception:
        use_transformer = False

    if use_transformer:
        try:
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model = AutoModelForSequenceClassification.from_pretrained(model_name)
            clf = TextClassificationPipeline(model=model, tokenizer=tokenizer,
                                             return_all_scores=True, truncation=True,
                                             max_length=max_len, device=-1)
        except Exception:
            # If model download fails (e.g., no internet), fall back to VADER
            use_transformer = False

    if not use_transformer:
        # VADER fallback
        try:
            from nltk.sentiment import SentimentIntensityAnalyzer
        except Exception:
            import nltk
            nltk.download('vader_lexicon', quiet=True)
            from nltk.sentiment import SentimentIntensityAnalyzer
        sia = SentimentIntensityAnalyzer()

    # ---------- Score reviews in batches ----------
    texts = reviews['comments_clean'].tolist()
    n = len(texts)

    p_pos = np.zeros(n, dtype=float)
    p_neu = np.zeros(n, dtype=float)
    p_neg = np.zeros(n, dtype=float)

    if use_transformer:
        # Helper to robustly map labels -> indices
        def scores_to_triple(score_list):
            # score_list = [{'label': 'negative', 'score': ...}, ...]
            out = {'pos':0.0,'neu':0.0,'neg':0.0}
            for d in score_list:
                lab = str(d['label']).lower()
                if 'pos' in lab: out['pos'] = float(d['score'])
                elif 'neu' in lab: out['neu'] = float(d['score'])
                elif 'neg' in lab or 'minus' in lab or lab.endswith('0'): out['neg'] = float(d['score'])
            # normalize if something odd
            s = out['pos'] + out['neu'] + out['neg']
            if s > 0:
                out = {k:v/s for k,v in out.items()}
            return out['pos'], out['neu'], out['neg']

        for i in range(0, n, batch_size):
            batch = texts[i:i+batch_size]
            res = clf(batch, batch_size=batch_size)
            # res is list (len=batch) of list-of-dicts
            for j, sc in enumerate(res, start=i):
                pp, pn, pg = scores_to_triple(sc)
                p_pos[j], p_neu[j], p_neg[j] = pp, pn, pg
    else:
        # VADER path: convert compound to 3-way probs with simple thresholds
        for i in range(n):
            c = sia.polarity_scores(texts[i])['compound']
            if c >= 0.05:
                p_pos[i], p_neu[i], p_neg[i] = 1.0, 0.0, 0.0
            elif c <= -0.05:
                p_pos[i], p_neu[i], p_neg[i] = 0.0, 0.0, 1.0
            else:
                p_pos[i], p_neu[i], p_neg[i] = 0.0, 1.0, 0.0

    # sentiment score in [-1, 1] similar to "pos - neg"
    score = p_pos - p_neg
    label_idx = np.argmax(np.c_[p_neg, p_neu, p_pos], axis=1)  # 0=neg,1=neu,2=pos
    labels = np.array(['negative','neutral','positive'])[label_idx]

    scored = reviews[['listing_id','date','comments_clean']].copy()
    scored['sentiment_score'] = score
    scored['label'] = labels
    scored['p_pos'] = p_pos
    scored['p_neu'] = p_neu
    scored['p_neg'] = p_neg

    # ---------- Aggregations ----------
    total = len(scored)
    overall = (scored['label'].value_counts()
               .reindex(['positive','neutral','negative'])
               .fillna(0).astype(int))
    overall_pct = (overall / max(1,total) * 100).round(2)

    listing_summary = (
        scored.groupby('listing_id').agg(
            n_reviews=('label','size'),
            pos_pct=('label', lambda s: (s=='positive').mean()*100),
            neu_pct=('label', lambda s: (s=='neutral').mean()*100),
            neg_pct=('label', lambda s: (s=='negative').mean()*100),
            avg_score=('sentiment_score','mean')
        ).round({'pos_pct':2,'neu_pct':2,'neg_pct':2,'avg_score':3})
        .sort_values('n_reviews', ascending=False)
    )

    monthly = scored.copy()
    monthly['month'] = monthly['date'].dt.to_period('M').dt.to_timestamp()
    monthly_trend = monthly.groupby('month')['sentiment_score'].mean().to_frame('mean_score')

    if verbose:
        print("\nOverall sentiment split (%):")
        print(overall_pct.dropna().to_string())
        # sample examples
        tops = scored.nlargest(3, 'sentiment_score')[['listing_id','date','sentiment_score','comments_clean']]
        bots = scored.nsmallest(3, 'sentiment_score')[['listing_id','date','sentiment_score','comments_clean']]
        print("\nTop positive examples:")
        print(tops.assign(comments_clean=lambda s: s['comments_clean'].str.slice(0,120)+'…').to_string(index=False))
        print("\nTop negative examples:")
        print(bots.assign(comments_clean=lambda s: s['comments_clean'].str.slice(0,120)+'…').to_string(index=False))

    # ---------- Plots ----------
    if plot:
        plt.figure()
        plt.hist(scored['sentiment_score'], bins=40, alpha=0.9)
        plt.title("Review sentiment score"); plt.xlabel("Score (-1 .. 1)"); plt.ylabel("Count")
        plt.tight_layout()

        plt.figure()
        order = ['positive','neutral','negative']
        bars = scored['label'].value_counts().reindex(order).fillna(0)
        plt.bar(bars.index.astype(str), bars.values)
        plt.title("Sentiment label distribution"); plt.ylabel("Reviews")
        plt.tight_layout()

        plt.figure()
        plt.plot(monthly_trend.index, monthly_trend['mean_score'])
        plt.title("Monthly mean sentiment score"); plt.xlabel("Month"); plt.ylabel("Mean score")
        plt.tight_layout()

    return {
        'reviews_scored': scored,
        'listing_summary': listing_summary,
        'monthly_trend': monthly_trend,
        'overall_pct': overall_pct
    }



: 

In [ ]:

# ---------- Filesystem helpers ----------
def _make_outdir(model_type, run_id):
    out = ART_ROOT / model_type / run_id
    out.mkdir(parents=True, exist_ok=True)
    return out

def save_current_figures(out_dir, prefix):
    saved = []
    for i, fignum in enumerate(plt.get_fignums(), start=1):
        fig = plt.figure(fignum)
        p = Path(out_dir) / f"{prefix}_{i}.png"
        fig.savefig(p, dpi=120, bbox_inches='tight')
        plt.close(fig)  # closes as we go
        saved.append(str(p))
    return saved

# ---------- Persistors for your three models ----------
def persist_regression_outputs(art_reg, params=None, dataset_id="airbnb_seattle"):
    """
    art_reg: dict returned by train_regression_hgb(...)
      keys used: 'model', 'metrics', 'feat_importance'(Series), 'test_predictions'(DataFrame), 'artifacts'
    """
    model_type = "regression"
    run_id = start_run(model_type, params=params, dataset_id=dataset_id)
    try:
        out = _make_outdir(model_type, run_id)

        # Save model
        model_path = out / "model.joblib"
        joblib.dump(art_reg['model'], model_path)
        log_artifact(run_id, model_path, 'model', 'Sklearn pipeline')

        # Save metrics JSON into DB (end_run handles it), also write a copy to disk
        metrics_path = out / "metrics.json"
        with open(metrics_path, 'w', encoding='utf-8') as f:
            json.dump(art_reg.get('metrics', {}), f, indent=2)
        log_artifact(run_id, metrics_path, 'text', 'Metrics JSON')

        # Save feature importance + test predictions
        if isinstance(art_reg.get('feat_importance'), pd.Series):
            fi_path = out / "feature_importance.parquet"
            art_reg['feat_importance'].rename("importance").to_frame().to_parquet(fi_path)
            log_artifact(run_id, fi_path, 'table', 'Permutation importance')

        if isinstance(art_reg.get('test_predictions'), pd.DataFrame):
            pred_path = out / "test_predictions.parquet"
            art_reg['test_predictions'].to_parquet(pred_path)
            log_artifact(run_id, pred_path, 'table', 'Test predictions')

        # Save any already-saved plots from the training function
        for p in art_reg.get('artifacts', {}).get('plots', []):
            if Path(p).exists():
                log_artifact(run_id, p, 'plot', 'Plot from training cell')

        # Also capture any currently open figures (e.g., if plot=True)
        for p in save_current_figures(out, prefix="reg_plot"):
            log_artifact(run_id, p, 'plot', 'Saved current matplotlib figure')
        
        end_run(run_id, 'succeeded', metrics=art_reg.get('metrics', {}))
        return run_id, str(out)
    except Exception as e:
        end_run(run_id, 'failed', notes=str(e))
        raise

def persist_logistic_outputs(clf, metrics: dict, params=None, dataset_id="airbnb_seattle"):
    """
    clf: Pipeline returned by run_logistic_price_bucket(...)
    metrics: dict returned by run_logistic_price_bucket(...)
    Captures the confusion matrix figure already drawn by your function.
    """
    model_type = "logistic"
    run_id = start_run(model_type, params=params, dataset_id=dataset_id)
    try:
        out = _make_outdir(model_type, run_id)

        # Save model
        model_path = out / "model.joblib"
        joblib.dump(clf, model_path)
        log_artifact(run_id, model_path, 'model', 'Logistic pipeline')

        # Save metrics
        metrics_path = out / "metrics.json"
        with open(metrics_path, 'w', encoding='utf-8') as f:
            json.dump(metrics or {}, f, indent=2)
        log_artifact(run_id, metrics_path, 'text', 'Metrics JSON')

        # Save any open figures (e.g., ConfusionMatrixDisplay)
        for p in save_current_figures(out, prefix="logistic_plot"):
            log_artifact(run_id, p, 'plot', 'Saved current matplotlib figure')
        
        end_run(run_id, 'succeeded', metrics=metrics or {})
        return run_id, str(out)
    except Exception as e:
        end_run(run_id, 'failed', notes=str(e))
        raise

def persist_kmeans_outputs(km_pipe, preview: pd.DataFrame, cluster_summary: pd.DataFrame,
                           metrics: dict, params=None, dataset_id="airbnb_seattle"):
    """
    km_pipe: Pipeline from run_kmeans_clusters(...)
    preview: small sample df
    cluster_summary: groupby summary
    metrics: {'silhouette': ...}
    """
    model_type = "kmeans"
    run_id = start_run(model_type, params=params, dataset_id=dataset_id)
    try:
        out = _make_outdir(model_type, run_id)

        # Save model
        model_path = out / "model.joblib"
        joblib.dump(km_pipe, model_path)
        log_artifact(run_id, model_path, 'model', 'KMeans pipeline')

        # Save tables
        if isinstance(preview, pd.DataFrame):
            prev_path = out / "preview.parquet"
            preview.to_parquet(prev_path)
            log_artifact(run_id, prev_path, 'table', 'Cluster preview')

        if isinstance(cluster_summary, pd.DataFrame):
            summ_path = out / "cluster_summary.parquet"
            cluster_summary.to_parquet(summ_path)
            log_artifact(run_id, summ_path, 'table', 'Cluster summary')

        # Save metrics
        metrics_path = out / "metrics.json"
        with open(metrics_path, 'w', encoding='utf-8') as f:
            json.dump(metrics or {}, f, indent=2)
        log_artifact(run_id, metrics_path, 'text', 'Metrics JSON')

        # Save any open figures (if you plotted something)
        for p in save_current_figures(out, prefix="kmeans_plot"):
            log_artifact(run_id, p, 'plot', 'Saved current matplotlib figure')

        end_run(run_id, 'succeeded', metrics=metrics or {})
        return run_id, str(out)
    except Exception as e:
        end_run(run_id, 'failed', notes=str(e))
        raise
def persist_forecast_outputs(art, params=None, dataset_id="airbnb_seattle",
                             horizon=None, future_days=None):
    """
    art: dict returned by run_calendar_forecast(...)
         keys used: 'model','metrics','daily','features','holdout_df','future_df'
    """
    assert all(k in art for k in ['model','metrics','daily','features','holdout_df','future_df']), \
        "Missing keys in forecast artifacts. Run run_calendar_forecast(...) first."

    model_type = "forecast"
    # fold horizon/future_days into params (if caller passes them)
    p = dict(params or {})
    if horizon is not None: p['horizon'] = int(horizon)
    if future_days is not None: p['future_days'] = int(future_days)

    run_id = start_run(model_type, params=p, dataset_id=dataset_id)
    try:
        out = _make_outdir(model_type, run_id)

        # --- Save model ---
        model_path = out / "model.joblib"
        joblib.dump(art['model'], model_path)
        log_artifact(run_id, model_path, 'model', 'HGB forecaster')

        # --- Save metrics JSON ---
        metrics_path = out / "metrics.json"
        with open(metrics_path, 'w', encoding='utf-8') as f:
            json.dump(art.get('metrics', {}), f, indent=2)
        log_artifact(run_id, metrics_path, 'text', 'Metrics JSON')

        # --- Save tables ---
        daily_path   = out / "daily.parquet"
        feats_path   = out / "features.parquet"
        holdout_path = out / "holdout.parquet"
        future_path  = out / "future.parquet"

        art['daily'].to_parquet(daily_path)
        art['features'].to_parquet(feats_path)
        art['holdout_df'].to_parquet(holdout_path)
        art['future_df'].to_parquet(future_path)

        log_artifact(run_id, daily_path,   'table', 'City daily series')
        log_artifact(run_id, feats_path,   'table', 'Feature matrix (lags/rolls)')
        log_artifact(run_id, holdout_path, 'table', 'Holdout predictions')
        log_artifact(run_id, future_path,  'table', 'Future forecast')

        # --- Save any open figures (train/test plots, forecast plot) ---
        for pth in save_current_figures(out, prefix="forecast_plot"):
            log_artifact(run_id, pth, 'plot', 'Saved matplotlib figure')

        end_run(run_id, 'succeeded', metrics=art.get('metrics', {}))
        return run_id, str(out)
    except Exception as e:
        end_run(run_id, 'failed', notes=str(e))
        raise


def persist_nlp_outputs(nlp_art, params=None, dataset_id="airbnb_seattle",
                        model_name=None, engine=None):
    """
    nlp_art: dict returned by run_review_sentiment_nlp(...)
             keys used: 'reviews_scored','listing_summary','monthly_trend','overall_pct'
    model_name/engine: optional metadata (e.g., HF model id or 'vader') to store with the run.
    """
    assert all(k in nlp_art for k in ['reviews_scored','listing_summary','monthly_trend','overall_pct']), \
        "Missing keys in NLP artifacts. Run run_review_sentiment_nlp(...) first."

    model_type = "nlp"
    meta = dict(params or {})
    if model_name is not None: meta['model_name'] = model_name
    if engine is not None:     meta['engine'] = engine

    run_id = start_run(model_type, params=meta, dataset_id=dataset_id)
    try:
        out = _make_outdir(model_type, run_id)

        # --- Save rollups/tables ---
        scored_path  = out / "reviews_scored.parquet"
        list_path    = out / "listing_summary.parquet"
        trend_path   = out / "monthly_trend.parquet"
        overall_path = out / "overall_pct.json"

        nlp_art['reviews_scored'].to_parquet(scored_path)
        nlp_art['listing_summary'].to_parquet(list_path)
        nlp_art['monthly_trend'].to_parquet(trend_path)
        with open(overall_path, 'w', encoding='utf-8') as f:
            json.dump({k: float(v) for k,v in nlp_art['overall_pct'].items()}, f, indent=2)

        log_artifact(run_id, scored_path,  'table', 'Per-review sentiment')
        log_artifact(run_id, list_path,    'table', 'Per-listing sentiment summary')
        log_artifact(run_id, trend_path,   'table', 'Monthly sentiment trend')
        log_artifact(run_id, overall_path, 'text',  'Overall sentiment split (%)')

        # --- Save any open figures (hist, bar, monthly plot) ---
        for pth in save_current_figures(out, prefix="nlp_plot"):
            log_artifact(run_id, pth, 'plot', 'Saved matplotlib figure')

        # --- Compose minimal metrics to store in runs table ---
        # Use overall split as metrics (handy for dashboard)
        metrics = {k: float(v) for k, v in nlp_art['overall_pct'].items()}
        end_run(run_id, 'succeeded', metrics=metrics)
        return run_id, str(out)
    except Exception as e:
        end_run(run_id, 'failed', notes=str(e))
        raise
# ---------- Minimal examples (call right after training) ----------
# 1) Regression:
art_reg = train_regression_hgb(listings_df=listings, reviews_df=reviews, plot=True, save_dir="artifacts/regression")
run_id_reg, out_reg = persist_regression_outputs(art_reg)
print("Saved regression run:", run_id_reg, "->", out_reg)

# 2) Logistic:
clf, cls_metrics = run_logistic_price_bucket()
run_id_cls, out_cls = persist_logistic_outputs(clf, cls_metrics)
print("Saved logistic run:", run_id_cls, "->", out_cls)

# 3) KMeans:
km_pipe, preview, cluster_summary, km_metrics = run_kmeans_clusters(n_clusters=5)
run_id_km, out_km = persist_kmeans_outputs(km_pipe, preview, cluster_summary, km_metrics)
print("Saved kmeans run:", run_id_km, "->", out_km)



# Forecast:
art = run_calendar_forecast(calendar_df=calendar, horizon=28, future_days=14, plot=True, verbose=True)
run_id_fc, out_fc = persist_forecast_outputs(art, params={'city':'Seattle'}, horizon=28, future_days=14)
print("Saved forecast run:", run_id_fc, "->", out_fc)

# NLP:
nlp_art = run_review_sentiment_nlp(reviews_df=reviews, plot=True, verbose=True)
run_id_nlp, out_nlp = persist_nlp_outputs(nlp_art, model_name="cardiffnlp/twitter-xlm-roberta-base-sentiment", engine="transformer")
print("Saved NLP run:", run_id_nlp, "->", out_nlp)

# Inspect history:
print(runs_history())
print(latest_run("regression"))

[HGB] MAE=$29.48  RMSE=$50.09  R²(log)=0.727

[Permutation importance] Top features:
cat__room_type_Entire home/apt    0.3848
num__accommodates                 0.0894
num__bedrooms                     0.0773
num__dist_to_spaceneedle_km       0.0700
num__bathrooms                    0.0509
num__availability_365             0.0451
cat__room_type_Private room       0.0346
num__dist_to_pikeplace_km         0.0271
num__host_since_days              0.0194
num__review_scores_rating         0.0116
num__minimum_nights               0.0106
num__host_listings_count          0.0075
num__number_of_reviews            0.0073
num__dist_to_center_km            0.0064
num__summary_len                  0.0060
cat__room_type_Shared room        0.0038
num__name_len                     0.0035
num__maximum_nights               0.0034
num__host_response_rate_num       0.0032
num__host_is_superhost_bin        0.0030
Saved regression run: 20251111-195133-regression-5317a26d -> artifacts\regression\20251111-1951

,count,mean_price,median_price
cluster,,,
1,1011,95.034619,85.0
4,660,101.393939,90.0
3,258,116.391473,100.0
2,1412,120.142351,109.0
0,477,264.031447,244.0


Saved kmeans run: 20251111-195148-kmeans-7bc7fd20 -> artifacts\kmeans\20251111-195148-kmeans-7bc7fd20
CALENDAR shape: (1393570, 6)
Missing (%):
 listing_id    0.0
date          0.0
available     0.0
dtype: float64
Duplicates (listing_id,date): 0
Date range: 2016-01-04 00:00:00 → 2017-01-02 00:00:00

Daily head:
             booked_count  total_listings  booked_share
date                                                  
2016-01-04          2083            3818      0.545574
2016-01-05          1951            3818      0.511001
2016-01-06          1992            3818      0.521739
2016-01-07          2042            3818      0.534835
2016-01-08          2036            3818      0.533263

Booked_share stats:
 count    365.000
mean       0.329
std        0.048
min        0.235
25%        0.298
50%        0.325
75%        0.353
max        0.546
Name: booked_share, dtype: float64

Holdout (28d) — Model MAE=38.21 RMSE=46.72 MAPE=3.90% | Naive MAE=21.79 RMSE=27.45 MAPE=2.20%
Saved forecas

: 

: 